In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import process

In [ ]:
animes = pd.read_csv('./animerecs/anime.csv')
ratings=pd.read_csv('./animerecs/rating.csv') 
#filter out ratings that are -1
ratings = ratings[ratings['rating'] != -1]

#create the user-item matrix
user_item_matrix = ratings.pivot_table(index='user_id', columns='anime_id', values='rating', fill_value=0)

# Handling NaN values. We fill them with 0s for now. This means the user hasn't rated that anime.
user_item_matrix = user_item_matrix.T

In [ ]:
#compute cosine similarity between users
item_similarity = cosine_similarity(user_item_matrix)

In [ ]:
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
print(item_similarity_df.head())

In [ ]:
def get_recs(anime_name, anime_data, similarity_matrix, num_recs=10):
    #fuzzy matching for potential typos
    match_tuple = process.extractOne(anime_name, anime_data['name'])
    
    if match_tuple[1] < 80:  # you can adjust this threshold
        return "No suitable match found. Please check the anime name and try again."
    
    title_match = match_tuple[0]
    
    #get the anime ID for the matched title
    anime_id = anime_data[anime_data['name'] == title_match]['anime_id'].values[0]
    
    #calculate the similarity scores for the given anime
    sim_scores = list(enumerate(similarity_matrix[anime_id]))
    
    #sort the animes based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #get the top, unique recommendations
    top_scores = sim_scores[1:num_recs+1]
    recommended_indices = [score[0] for score in top_scores]
    unique_animes = anime_data.iloc[recommended_indices]['name'].unique()
    
    return unique_animes

In [ ]:
#assuming you have loaded your anime data and similarity matrix
user_input = input("Enter your favorite anime: ")
recommendations = get_recs(user_input, animes, item_similarity)

if isinstance(recommendations, str):
    print(recommendations)
else:
    print("Based on your favorite anime, we recommend:\n")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")